In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns

import gc
import warnings
import os
import time

from contextlib import contextmanager
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import KFold, StratifiedKFold

#warnings.simplefilter(action='ignore', category=FutureWarning)

print(os.listdir("../Projet+Mise+en+prod+-+home-credit-default-risk"))

@contextmanager
def timer(title):
    t0 = time.time()
    yield
    print("{} - done in {:.0f}s".format(title, time.time() - t0))



['application_test.csv', '.DS_Store', 'HomeCredit_columns_description.csv', 'POS_CASH_balance.csv', 'credit_card_balance.csv', 'installments_payments.csv', 'application_train.csv', 'bureau.csv', 'previous_application.csv', 'bureau_balance.csv', 'sample_submission.csv']


In [ ]:
# Training data
app_train = pd.read_csv('../my_csv_files/MY_df_for_pycaret.csv')
print('Training data shape: ', app_train.shape)
app_train.head()

In [3]:
import re

app_train = app_train.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
train_df = app_train[app_train['TARGET'].notnull()]
test_df = app_train[app_train['TARGET'].isnull()]
#feats = [f for f in train_df.columns if f not in ['TARGET','SK_ID_CURR','SK_ID_BUREAU','SK_ID_PREV','index']]
feats = [f for f in train_df.columns if f not in ['SK_ID_CURR','SK_ID_BUREAU','SK_ID_PREV',
                                                  'Unnamed: 0',
                                                  'index']]
train_df[feats]
train_df['TARGET']

0         1.0
1         0.0
2         0.0
3         0.0
4         0.0
         ... 
307502    0.0
307503    0.0
307504    0.0
307505    1.0
307506    0.0
Name: TARGET, Length: 307507, dtype: float64

## Classification with PyCaret

In [4]:
#import classification module 
from pycaret.classification import * 

#intialize the setup (in Notebook env)
exp_clf = setup(train_df[feats], target = 'TARGET', fold_shuffle = True,
                train_size = 0.7, data_split_shuffle=True, session_id = 2)
                #imputation_type='iterative')

#intialize the setup (in Non-Notebook env)
#exp_clf = setup(train_df[feats], target = 'TARGET', html = False)

#intialize the setup (remote runs like Kaggle or GitHub actions)
#exp_clf = setup(train_df[feats], target = 'TARGET', silent = True)

,Description,Value
0,session_id,2
1,Target,TARGET
2,Target Type,Binary
3,Label Encoded,"0.0: 0, 1.0: 1"
4,Original Data,"(307507, 797)"
5,Missing Values,True
6,Numeric Features,563
7,Categorical Features,233
8,Ordinal Features,False
9,High Cardinality Features,False


In [7]:
compare_models(exclude = ["knn", "rf", "ada", "ridge", "qda", "catboost", "et", "lda"])

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
gbc,Gradient Boosting Classifier,0.9197,0.7720,0.0215,0.5566,0.0415,0.0357,0.0977,311.8660
lightgbm,Light Gradient Boosting Machine,0.9197,0.7811,0.0364,0.5402,0.0683,0.0587,0.1247,14.7730
xgboost,Extreme Gradient Boosting,0.9185,0.7720,0.0628,0.4614,0.1105,0.0930,0.1472,1575.4490
svm,SVM - Linear Kernel,0.8650,0.0000,0.0746,0.0971,0.0708,0.0089,0.0108,60.7150
dt,Decision Tree Classifier,0.8548,0.5424,0.1699,0.1492,0.1589,0.0798,0.0800,864.2130
nb,Naive Bayes,0.2178,0.5648,0.8766,0.0839,0.1532,0.0069,0.0274,4.4430
lr,Logistic Regression,0.0919,0.0586,0.0001,0.0100,0.0001,0.0000,0.0002,698.9290


GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=2, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [ ]:
# for other second pycaret notebook
#compare_models(include = ["gbc", "lightgbm", "ada", "ridge", "svm", "nb"])